In [ ]:
import torch
from PIL import Image
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from google.colab import drive
import os
import random
import shutil
from torchvision import models


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_PATH = '/content/drive/MyDrive/AI_Real_Dataset/raw/'

src_dir = {
    'ai': os.path.join(BASE_PATH, 'generatedAI'),
    'real': os.path.join(BASE_PATH, 'realPic/pic')
}

dataset = {
    'train': {
            'ai': os.path.join(BASE_PATH, 'train/ai'),
            'real': os.path.join(BASE_PATH, 'train/real')
    },
        'validation': {
            'ai': os.path.join(BASE_PATH, 'validation/ai'),
            'real': os.path.join(BASE_PATH, 'validation/real')
    },
        'test': {
            'ai': os.path.join(BASE_PATH, 'test/ai'),
            'real': os.path.join(BASE_PATH, 'test/real')
    }
}


SPLIT = (0.7, 0.15, 0.15)

In [ ]:
!echo "TRAIN AI:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/train/ai | wc -l
!echo "VAL AI:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/validation/ai | wc -l
!echo "TEST AI:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/test/ai | wc -l

!echo "TRAIN REAL:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/train/real | wc -l
!echo "VAL REAL:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/validation/real | wc -l
!echo "TEST REAL:" && ls /content/drive/MyDrive/AI_Real_Dataset/raw/test/real | wc -l

TRAIN AI:
11349
VAL AI:
2431
TEST AI:
2433
TRAIN REAL:
11349
VAL REAL:
2431
TEST REAL:
2433


In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
!mkdir -p /content/dataset/raw

In [ ]:
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/train /content/dataset/raw/
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/test /content/dataset/raw/
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/validation /content/dataset/raw/

In [ ]:
!echo "TRAIN AI:" && ls /content/dataset/raw/train/ai | wc -l
!echo "VAL AI:" && ls /content/dataset/raw/validation/ai | wc -l
!echo "TEST AI:" && ls /content/dataset/raw/test/ai | wc -l

!echo "TRAIN REAL:" && ls /content/dataset/raw/train/real | wc -l
!echo "VAL REAL:" && ls /content/dataset/raw/validation/real | wc -l
!echo "TEST REAL:" && ls /content/dataset/raw/test/real | wc -l

TRAIN AI:
ls: cannot access '/content/dataset/raw/train/ai': No such file or directory
0
VAL AI:
ls: cannot access '/content/dataset/raw/validation/ai': No such file or directory
0
TEST AI:
2433
TRAIN REAL:
ls: cannot access '/content/dataset/raw/train/real': No such file or directory
0
VAL REAL:
ls: cannot access '/content/dataset/raw/validation/real': No such file or directory
0
TEST REAL:
2433


In [ ]:
train_dataset = datasets.ImageFolder(root='/content/dataset/raw/train', transform=transform_train)
validation_dataset = datasets.ImageFolder(root='/content/dataset/raw/validation', transform=transform_val)
test_dataset = datasets.ImageFolder(root='/content/dataset/raw/test', transform=transform_test)

In [ ]:
test_dataset = datasets.ImageFolder(root='/content/dataset/raw/test', transform=transform_test)

In [ ]:
test_dataset.class_to_idx

{'ai': 0, 'real': 1}

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
images, labels = next(iter(test_loader))
print(images.shape)
print(labels)

torch.Size([32, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
device=''
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

print(device)

cuda


In [ ]:
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 1)
model = model.to(device)


In [ ]:
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier[1].parameters():
  param.requires_grad = True

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=1e-4)

In [ ]:
scaler = torch.amp.GradScaler()

In [ ]:
def train(model, loader, optimizer, criterion):

    model.train()
    train_loss = 0.0

    for i, (inputs, labels) in enumerate(loader):

        inputs = inputs.to(device)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * inputs.size(0)

    return train_loss / len(loader.dataset)

In [ ]:
def validate(model, loader, criterion):

    model.eval()
    running_loss  = 0.0
    correct = 0
    total = 0

    with torch.no_grad():

        for i, (inputs, labels) in enumerate(loader):

            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)


            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            correct += (preds.squeeze(1) == labels.squeeze(1)).sum().item()
            total += labels.size(0)
            running_loss += loss.item() * inputs.size(0)


    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
epochs = 5
PATIENCE = 2
best_val_acc = 0.0
patience_counter = 0

for epoch in range(epochs):

    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, validation_loader, criterion)

    print(f"""Epoch [{epoch+1}/{epochs}]
            Train Loss: {train_loss:.4f}
            Val Loss:   {val_loss:.4f}
            Val Acc:    {val_acc:.4f}
    """)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        print("Early stopping triggered")
        break

Epoch [1/5]
            Train Loss: 0.4667
            Val Loss:   0.3336
            Val Acc:    0.9284
    
Epoch [2/5]
            Train Loss: 0.2869
            Val Loss:   0.2414
            Val Acc:    0.9449
    
Epoch [3/5]
            Train Loss: 0.2293
            Val Loss:   0.1994
            Val Acc:    0.9517
    
Epoch [4/5]
            Train Loss: 0.1992
            Val Loss:   0.1761
            Val Acc:    0.9548
    
Epoch [5/5]
            Train Loss: 0.1823
            Val Loss:   0.1631
            Val Acc:    0.9517
    


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/AI_Real_Dataset/pretrainedRestnet/efficientNet_B0_Head.pth")

In [ ]:
for p in model.parameters():
    p.requires_grad = False
for p in model.features[-1].parameters():
    p.requires_grad = True
for p in model.classifier[1].parameters():
    p.requires_grad = True

In [ ]:
optimizer = torch.optim.AdamW(
    [
        {"params": model.features[-1].parameters(), "lr": 1e-5},
        {"params": model.classifier[1].parameters(),     "lr": 1e-4},
    ],
    weight_decay=1e-4
)

criterion = nn.BCEWithLogitsLoss()


In [ ]:
EPOCHS = 10
PATIENCE = 3
best_val_acc = 0.0
patience_counter = 0


for epoch in range(EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, validation_loader, criterion)

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        print("Early stopping triggered")
        break

Epoch [1/10] | Train Loss: 0.1567 | Val Loss: 0.1385 | Val Acc: 0.9589
Epoch [2/10] | Train Loss: 0.1366 | Val Loss: 0.1210 | Val Acc: 0.9634
Epoch [3/10] | Train Loss: 0.1238 | Val Loss: 0.1045 | Val Acc: 0.9677
Epoch [4/10] | Train Loss: 0.1176 | Val Loss: 0.0933 | Val Acc: 0.9700
Epoch [5/10] | Train Loss: 0.1065 | Val Loss: 0.0864 | Val Acc: 0.9702
Epoch [6/10] | Train Loss: 0.1030 | Val Loss: 0.0810 | Val Acc: 0.9733
Epoch [7/10] | Train Loss: 0.0990 | Val Loss: 0.0802 | Val Acc: 0.9722
Epoch [8/10] | Train Loss: 0.0909 | Val Loss: 0.0759 | Val Acc: 0.9747
Epoch [9/10] | Train Loss: 0.0901 | Val Loss: 0.0735 | Val Acc: 0.9759
Epoch [10/10] | Train Loss: 0.0856 | Val Loss: 0.0722 | Val Acc: 0.9763


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/AI_Real_Dataset/pretrainedRestnet/efficientNet_B0_Head_MBConv.pth")

In [ ]:
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/test /content/dataset/raw/

In [ ]:
model = efficientnet_b0(weights=None)

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 1)
model = model.to(device)

In [ ]:
ckpt_path = '/content/drive/MyDrive/AI_Real_Dataset/pretrainedRestnet/efficientNet_B0_Head_MBConv.pth'
model.load_state_dict(torch.load(ckpt_path, map_location=device))
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
test_loss, test_acc = validate(
    model,
    test_loader,
    criterion
)

In [ ]:
print(f"TEST Loss: {test_loss:.4f}")
print(f"TEST Accuracy: {test_acc:.4f}")

TEST Loss: 0.0772
TEST Accuracy: 0.9716


In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/AI_Real_Dataset/pretrainedRestnet/efficientNet_BO_Final.pth"
)